# Scraping Tweets from a public Twitter account between two dates

snscraper has a very easy to use python wrapper than can be used to scrape tweets from a public Twitter account.

In [79]:
from tqdm import tqdm
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import date, timedelta
from scrape_utils import scrape_twitter, scrape_twitter_users

In [ ]:
scrape_twitter(query="PLDT")

In [81]:
df = pd.read_csv("tweets_PLDT.csv")
df.head()

,tweet_id,date,user_name,tweet,retweets,likes,quote_tweets,replies,retweeted_tweet,quoted_tweet
0,1538415754676432896,2022-06-19 06:57:52+00:00,heyymarierawrrr,AALIS NA AKO NG PINAS AYOKO NA DITO LILIPAT NA...,0,0,0,0,NaN,NaN
1,1538415653471899649,2022-06-19 06:57:28+00:00,PLDT_Cares,"@dgetgogirl What seems to be the problem, @dge...",0,0,0,0,NaN,NaN
2,1538415473393815552,2022-06-19 06:56:45+00:00,sevithethird,Ang lala ng internet namin ngayon @PLDT_Cares ...,0,0,0,0,NaN,NaN
3,1538415383077863424,2022-06-19 06:56:24+00:00,NSKALATS,kagabi pa dapat ‘to kaso bulok pldt. https://t...,0,0,0,0,NaN,https://twitter.com/NSKALATS/status/1538415226...
4,1538414813025488897,2022-06-19 06:54:08+00:00,sunprincehyuck,@PLDT_Cares hi mayroon bang maintenance sa iba...,0,0,0,0,NaN,NaN


In [ ]:
users = df.user_name.to_list()
scrape_twitter_users(users, filename="PLDT")

In [83]:
df_users = pd.read_csv("tweets_PLDT_users.csv")
df_users.head()

,user_id,user_name,display_name,description,verified,created,followers,following,tweets,location
0,1385098242191335427,heyymarierawrrr,🍑,@BLACKPINK 's 💗,False,2021-04-22 05:09:06+00:00,124,248,1309,jl & cs
1,1526439727,PLDT_Cares,PLDT Cares,Welcome to the official customer support accou...,True,2013-06-18 01:48:56+00:00,217399,88920,788386,Philippines
2,241039494,sevithethird,Sevi Guevara III,It's just that he was all alone. Always by him...,False,2011-01-21 09:19:26+00:00,91,118,14788,Lungsod Quezon
3,1468257883578318848,NSKALATS,ً,NaN,False,2021-12-07 16:35:41+00:00,11,12,111,7 — tatlong gwapo\nat apat na pangit ⊹
4,1169761912105533440,sunprincehyuck,🌻,"lee donghyuck, the sun prince ☀️",False,2019-09-05 23:59:25+00:00,0,28,1106,"‘95, for bns and reading"
